# 2D / 3D Mapping

 ## Boreholes 2D mapping from files

### imports

In [2]:
import matplotlib.pyplot as plt
#from striplog import Component, Decor, Legend
#from core.omf import Borehole3D, striplog_legend_to_omf_legend 

import geopandas as gpd, pandas as pd
from shapely import wkt
import folium as fm
from folium import plugins
from utils.io import boreholes_from_files, striplog_from_text, export_gdf, read_gdf_file
from os import remove
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

### Data from a shapefile

In [3]:
shapef_dir = ROOT_DIR+"/CF_data/Data_UMONS/SIG/"
shp = gpd.read_file(shapef_dir+"Pilote_v8.0.shp")

**Rmq** : ici à quoi correspondent :

    - ID (tout est à 0!), 
    - Ref
    - Type

In [4]:
if 'Id' in shp.columns:
    shp=shp.drop(['Id'], axis=1)
    #shp.index.name='Id'
shp.head(2)

,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.891 122592.127)
1,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.447 122588.102)


In [5]:
pwd


'/home/yanathan/Projects/GSDMA/playground/TFE_test'

### Save location data in a Geopackage / GeoJson / csv

In [6]:
export_gdf(shp, 31370, 'tmp_files/test_loc.csv')

tmp_files/test_loc.csv has been saved !


In [7]:
cat 'tmp_files/test_loc.csv'

X,Y,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
152894.89100002544,122592.12713904586,1,1,111,2,1,1,0,1,0,OUI,POINT (152894.8910000254 122592.1271390459)
152881.4470000254,122588.10213904455,1,1,120,2,1,1,0,1,0,OUI,POINT (152881.4470000254 122588.1021390446)
152885.99500002537,122587.12713904679,2,1,217,3,1,1,1,1,0,OUI,POINT (152885.9950000254 122587.1271390468)
152883.7680000254,122589.75413904712,1,1,116,2,1,1,0,1,0,OUI,POINT (152883.7680000254 122589.7541390471)
152888.1280000254,122588.43613904715,2,1,214,2,1,1,1,1,0,OUI,POINT (152888.1280000254 122588.4361390471)
152886.2260000254,122591.65513904672,1,1,113,2,1,1,0,1,0,OUI,POINT (152886.2260000254 122591.6551390467)
152880.9800000254,122583.69313904643,2,1,223,3,1,1,1,1,1,OUI,POINT (152880.9800000254 122583.6931390464)
152883.65800002537,122585.62313904893,2,1,221,2,1,1,1,1,1,OUI,POINT (152883.6580000254 122585.6231390489)
152889.1520000254,122587.1761390483,1,1,118,2,1,1,0,1,0,OUI,POINT (152889.15200002

### 2D Map

-idée à explorer : fonction pour générer une carte des forages présents dans un projet

In [8]:
#Change CRS EPSG 31370 (Lambert 72) into EPSG 4326 (WGS 84)
shp = shp.to_crs(epsg=4326)
center=[shp.geometry.y.mean(), shp.geometry.x.mean()]

# Use a satellite map
tile_name = ["Satellite"]
tile_attr = ["Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"]
tile_url = ["https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"]

BH_map=fm.Map(location=center, tiles='OpenStreetMap', zoom_start=19, max_zoom=25, control_scale=True)

ch1 = fm.FeatureGroup(name='Boreholes')

for idx, row in shp.iterrows():
    fm.CircleMarker([row.geometry.y, row.geometry.x], popup=str(idx) , radius=0.1,
                     color='red', fill_color='red',
                    opacity=0.6
                    ).add_to(ch1)
    
#fm.GeoJson('BH_loc.json').add_to(fm_map)

mini_map = plugins.MiniMap(toggle_display=True, zoom_level_offset=-6)

#adding features to the base_map
fm.TileLayer(name=tile_name[0], tiles=tile_url[0], max_zoom=25, attr=tile_attr[0], control=True).add_to(BH_map)
ch1.add_to(BH_map)
fm.LayerControl().add_to(BH_map)
BH_map.add_child(mini_map)

#save in a file
BH_map.save('tmp_files/BH_location.html')
BH_map

### Reading location data from a csv, Geo(package/Json) file

In [9]:
file_dir=ROOT_DIR+'/playground/TFE_test/tmp_files/'

In [10]:
bh_loc=read_gdf_file(file_dir+'test_loc.csv', 31370, 4326)
bh_loc.head(2)

,X,Y,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
0,4.409477,50.414208,1,1,111,2,1,1,0,1,0,OUI,POINT (4.40948 50.41421)
1,4.409288,50.414172,1,1,120,2,1,1,0,1,0,OUI,POINT (4.40929 50.41417)


**Rmq** : la coordonnée Z est manquante dans les données de base !

### Create a boreholes positions list

In [12]:
x_list=bh_loc['X'].to_list()
y_list=bh_loc['Y'].to_list()
len(x_list)

88

## Borehole 3D mapping

### Build Borehole3D objects

Puisqu'on ne dispose que de la localisation des forages (pas d'infos sur la géométrie et le "contenu" des forages), l'idée est de créer **une géométrie et un contenu par défaut** pour chaque forage, **tout en respectant les positions**, puis les enregistrer dans la BD afin de pouvoir les afficher tous dans l'environnement 3D. La mise à jour des données manquantes se fera plus tard.

In [ ]:
remove('tmp_files/tfe_orm_db.db')

Create the db engine and the database structure

In [ ]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)
Base.metadata.create_all(engine)

creation of the Boreholes from las, csv files

In [ ]:
lexicon = Lexicon.default() #to do : create our own lexicon

In [ ]:
borehole_dict = {'F1':ROOT_DIR+'/data/test.las'}#, 'F2':ROOT_DIR+'/data/test.csv'} # boreholes to insert into the db

In [ ]:
boreholes, components= boreholes_from_files(borehole_dict, x=x_list, y=y_list, verbose=0)

Create the session and the project

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

p = Project(session)
p.add_components(components)

Add boreholes into the database

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.commit()

In [ ]:
p.boreholes[0].intervals[0].description

In [ ]:
session.close()

In [ ]:
p.plot3d(x3d=True, radius=0.5)